# INBOX

See all incoming messages in this notebook. Everyone can log in with their access data and assign this notebook to their HMI. To do this, simply enter your own HMI with the corresponding secret as the client in this script. Running this notebook, you will authorize your HMI to call up the wheel loader's location. 
Just go to the **Cell** drop-down menu and use the **Run All** button.

First, all necessary modules are imported into the script, including the S³I library.

In [1]:
import s3i
import time 
import base64 
import getpass
import receiver
import pykka
import jwt
import json
import logging
logging.basicConfig(level=logging.ERROR)

This Notebook acts as a client in terms of the S³I. In order to use the S³I this notebook needs a client id and the respective secret. You can assign this notebook to your personal HMI, to make this notebook your HMI. Therefore enter the id and the secret of your HMI in the following input fields.

In [2]:
CLIENT = "s3i:0a884197-1314-440b-920a-f1c63b46201d" #None
SECRET = "257aa70b-cc7d-43c3-9f27-ea99caafd119" #getpass.getpass('[S3I]: Please enter the secret:') print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) +
 #         "]: Assign a client to this notebook. (The id of your HMI)")
clientId = input('[S3I]: Please enter the client id:')
clientSecret = getpass.getpass('[S3I]: Please enter the secret:')
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) +
          "]: Client id and secret are set")

[S3I]: Please enter the client id:s3i:630f063f-0eab-4dd3-97d4-da2634705e51
[S3I]: Please enter the secret:········
[S3I][2020-05-24 20:24:53]: Client id and secret are set


Next, you have to enter your username and password. With your access data a token is requested which authorizes this client (your HMI) to call up the wheel loader's location on your behalf.

In [3]:
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) +
          "]: DEMO Wheel loader (service call), please log in! (sachverstaendiger)")
username = input('[S3I]: Please enter your username:')
password = getpass.getpass('[S3I]: Please enter the password:')
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                                   ) + "]: Your credentials are sent to S3I IdentityProvider.")
s3i_identity_provider = s3i.IdentityProvider(grant_type='password', identity_provider_url="https://idp.s3i.vswf.dev/", realm='KWH',
                                         client_id=clientId, client_secret=clientSecret, username=username, password=password)
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN, scope="")

''' decode the access token
'''
parsed_username = jwt.decode(access_token, verify=False)[
    "preferred_username"]

print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                               time.localtime()) + "]: Token received " + parsed_username + " logged in.")

[S3I][2020-05-24 20:24:53]: DEMO Wheel loader (service call), please log in! (sachverstaendiger)
[S3I]: Please enter your username:test_workshop
[S3I]: Please enter the password:········
[S3I][2020-05-24 20:25:05]: Your credentials are sent to S3I IdentityProvider.
[S3I][2020-05-24 20:25:06]: Token received test_workshop logged in.


In [5]:
class MessageEncryptor(pykka.ThreadingActor):
       
    def callback(self, pgp_message):
        print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                                   time.localtime()) + "]: A new message has been received")
        message_str = pgp_message.replace("\\n", "\n").strip('"') 
        uMsg = s3i.UserMessage(msg_blob=message_str)
    
        access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
        dir = s3i.Directory(s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)
        """Decrypt the message and verify the signature
        """
        if uMsg.pgpMsg.is_encrypted:
            sec_key = s3i.Key(path_demo_dir="key/", filename="secKey.asc")
            uMsg.decryptAndVerify(sec_key, "", dir)
            #uMsg.decrypt(sec_key.key)
            uMsg.convertPgpToMsg()

        print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                                   time.localtime()) + "]: Subject of the message: " + uMsg.msg["subject"])
        print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                                   time.localtime()) + "]: Text of the message:  " + uMsg.msg["text"])
        print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                                   time.localtime()) + "]: Sender of the message:  :", dir.queryThingIDBased(uMsg.msg["sender"]+"/attributes/name"))

        attachments_list = uMsg.msg["attachments"]
        """
        store the attachment file in specified path
        """
        for attachment in attachments_list:
            with open("received_data/"+attachment["filename"], 'wb') as file:
                decode = base64.b64decode(attachment["data"])
                file.write(decode)
                print("[S3I]: Attachment " + attachment["filename"]
                   + " of the message is stored in received_data")
                            
        print("==============================================================================")

                            
messageEncryptor = MessageEncryptor.start().proxy()

In [6]:
s3i_receiver = receiver.Receiver.start("s3ibs://"+clientId,s3i_identity_provider,messageEncryptor).proxy()
s3i_receiver.start_receiving()

In actor
[S3I][2020-05-24 20:25:06]: A new message has been received
[S3I][2020-05-24 20:25:06]: The signature is valid.
[S3I][2020-05-24 20:25:06]: Decryption successful
[S3I][2020-05-24 20:25:06]: Subject of the message: Hello
[S3I][2020-05-24 20:25:06]: Text of the message:  oh mann!!
[S3I][2020-05-24 20:25:06]: Sender of the message:  : {'status': 404, 'error': 'things:attribute.notfound', 'message': "The attribute with key '/name' on the thing with ID 's3i:630f063f-0eab-4dd3-97d4-da2634705e51' could not be found or the requester had insufficient permissions to access it.", 'description': 'Check if the ID of the thing and the key of your requested attribute was correct and you have sufficient permissions.'}
In actor
[S3I][2020-05-24 20:25:20]: A new message has been received
[S3I][2020-05-24 20:25:20]: The signature is valid.
[S3I][2020-05-24 20:25:20]: Decryption successful
[S3I][2020-05-24 20:25:20]: Subject of the message: Hello
[S3I][2020-05-24 20:25:20]: Text of the message:  